# 1. Import & Read

In [1]:
import pandas as pd
import os
import torch
from keybert import KeyBERT
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
from sentence_transformers import SentenceTransformer

from langchain_core.messages import ChatMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.embeddings import Embeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_community.document_loaders import TextLoader, PyPDFLoader, PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

from langchain_openai import ChatOpenAI

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langserve.pydantic_v1 import BaseModel, Field
from typing import List, Union

from openai import OpenAI
import os
import warnings
warnings.filterwarnings("ignore")

pdf_dir = "file/rag_score_pdf/"

score_df = pd.read_csv("file/rag_evaluation_result.csv")

/Users/shinminseog/opt/anaconda3/envs/dl/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# 2. 답변 생성

In [2]:
RAG_PROMPT_TEMPLATE = (
    """
    You always answer into Korean. You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
    Use all the sentences and keep the answer concise in bullet.\nQuestion: {question} \nContext: {context} \nAnswer:
    """
)

SYS_PROMPT_TEMPLATE = (
    """
    당신은 ** 에이전트입니다. 가능한 한 도움이 될 수 있도록 사실적인 정보만을 제공하도록 설계되었습니다. 
    친절하게 응대하지만 지나치게 수다스럽지는 않습니다. 아래에 제공된 컨텍스트 정보를 바탕으로, 사전 지식 없이 질문에 답하십시오.
    """
)

## 2-1) 토큰화 X

In [3]:
# PDF 로더 함수
def load_pdf(file_path):
    loader = PyMuPDFLoader(pdf_dir + file_path)
    return loader.load()

# 텍스트 분할 함수
def split_text(pages, tokenizer):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=256,
        chunk_overlap=20,
        length_function=lambda text: len(tokenizer.encode(text)),
    )
    return text_splitter.split_documents(pages)

# Embeddings 클래스 정의
class MyEmbeddings(Embeddings):
    def __init__(self, model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode(texts, convert_to_tensor=True).tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]

# RAG 시스템 설정 함수
def setup_rag_system(llm_base_url, embedding_model_name):
    # 언어 모델 설정
    llm = ChatOpenAI(
        base_url=llm_base_url,
        api_key="lm-studio",
        model="teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
        temperature=0.1,
    )
    # 임베딩 모델 설정
    emb = MyEmbeddings(model_name=embedding_model_name)

    # RAG 체인 설정
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", "You always answer into Korean."),
        ("user", RAG_PROMPT_TEMPLATE),
    ])
    rag_chain = rag_prompt | llm | StrOutputParser()

    return llm, emb, rag_chain

# 문서 저장소 및 검색기 설정 함수
def setup_vectorstore(docs, emb):
    vectorstore = FAISS.from_documents(docs, embedding=emb, distance_strategy=DistanceStrategy.COSINE)
    return vectorstore.as_retriever(search_kwargs={'k': 3})

# 질문 요약 함수
def summarize_question(question: str, tokenizer, bart_model) -> str:
    inputs = tokenizer.encode(question, return_tensors="pt")
    summary_ids = bart_model.generate(inputs, max_length=200, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 키워드 추출 함수
def extract_keywords(text: str, keyword_model, top_n: int = 5) -> List[str]:
    return [keyword[0] for keyword in keyword_model.extract_keywords(text, top_n=top_n)]

# 문서 포맷팅 함수
def format_docs(docs):
    return " ".join(doc.page_content for doc in docs)

# 질문 처리 함수
def process_question(question: str, retriever, tokenizer, bart_model, keyword_model, chain):
    summarized_question = summarize_question(question, tokenizer, bart_model)
    keywords = extract_keywords(question, keyword_model)
    retriever_result = retriever.get_relevant_documents(" ".join(keywords))
    context = format_docs(retriever_result)

    input_data = {
        "context": context,
        "question": summarized_question
    }

    return chain.invoke(input_data)

# 전체 프로세스 실행 함수
def run_pipeline(file_path, question, llm_base_url="http://localhost:1234/v1"):
    # PDF 로드
    pages = load_pdf(file_path)

    # KoBART 모델 및 토크나이저 로드
    tokenizer = PreTrainedTokenizerFast.from_pretrained("EbanLee/kobart-summary-v3")
    bart_model = BartForConditionalGeneration.from_pretrained("EbanLee/kobart-summary-v3")

    # 텍스트 분할
    docs = split_text(pages, tokenizer)

    # RAG 시스템 및 체인 설정
    llm, emb, rag_chain = setup_rag_system(llm_base_url, "bespin-global/klue-sroberta-base-continue-learning-by-mnr")

    # 문서 저장소 및 검색기 설정
    retriever = setup_vectorstore(docs, emb)

    # KeyBERT 모델 로드
    keyword_model = KeyBERT()

    # 질문 처리 및 답변 생성
    return process_question(question, retriever, tokenizer, bart_model, keyword_model, rag_chain)

## 2-2) 토큰화 O

In [ ]:
# PDF 로더 함수
def load_pdf(file_path):
    loader = PyMuPDFLoader(pdf_dir + file_path)
    pages = loader.load()
    return pages

# 텍스트 분할 함수
def split_text(pages, tokenizer):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=256,
        chunk_overlap=20,
        separators=["\n\n", "\n", "(?<=\. )", " ", ""],
        length_function=lambda text: len(tokenizer.encode(text)),
    )
    docs = text_splitter.split_documents(pages)
    return docs

# 텍스트 클리닝 함수
def clean_text(text):
    return ' '.join(text.split())

# 문서 토크나이징 함수
def tokenize_docs(docs, tokenizer):
    tokenized_docs = []
    for doc in docs:
        tokenized_text = tokenizer(clean_text(doc.page_content), return_tensors="pt")
        tokenized_docs.append(tokenized_text)
    return tokenized_docs

# Embeddings 클래스 정의
class MyEmbeddings(Embeddings):
    def __init__(self, model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        embeddings = self.model.encode(texts, convert_to_tensor=True)
        return embeddings.tolist()

    def embed_query(self, text: str) -> List[float]:
        embedding = self.model.encode([text], convert_to_tensor=True)
        return embedding[0].tolist()

# RAG 시스템 설정 함수
def setup_rag_system(llm_base_url, embedding_model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr"):
    # 언어 모델 설정
    llm = ChatOpenAI(
        base_url=llm_base_url,
        api_key="lm-studio",
        model="teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
        temperature=0.1,
    )

    # 임베딩 모델 설정
    emb = MyEmbeddings(model_name=embedding_model_name)

    # RAG 체인 설정
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", "You always answer into Korean."),
        ("user", "Context: {context}\n\nQuestion: {question}\n\nAnswer:"),
    ])
    rag_chain = rag_prompt | llm | StrOutputParser()

    return llm, emb, rag_chain

# 문서 저장소 및 검색기 설정 함수
def setup_vectorstore(docs, emb):
    vectorstore = FAISS.from_documents(docs, embedding=emb, distance_strategy=DistanceStrategy.COSINE)
    retriever = vectorstore.as_retriever(search_kwargs={'k': 3})
    return vectorstore, retriever

# 질문 요약 함수
def summarize_question(question: str, tokenizer, bart_model) -> str:
    inputs = tokenizer.encode(question, return_tensors="pt")
    summary_ids = bart_model.generate(inputs, max_length=200, min_length=0, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# 키워드 추출 함수
def extract_keywords(text: str, keyword_model, top_n: int = 5) -> List[str]:
    keywords = keyword_model.extract_keywords(text, top_n=top_n)
    return [keyword[0] for keyword in keywords]

# 문서 포맷팅 함수
def format_docs(docs):
    return " ".join(doc.page_content for doc in docs)

# 질문 처리 함수
def process_question(question: str, retriever, tokenizer, bart_model, keyword_model, chain, docs):
    summarized_question = summarize_question(question, tokenizer, bart_model)
    keywords = extract_keywords(question, keyword_model)
    retriever_result = retriever.get_relevant_documents(" ".join(keywords))
    context = format_docs(retriever_result)

    # 토크나이징된 문서 활용
    tokenized_contexts = tokenize_docs(docs, tokenizer)
    # 토크나이징된 문서로 context 업데이트
    # 예시: 단순히 첫 번째 토크나이징된 문서를 사용
    if tokenized_contexts:
        context += " " + tokenizer.decode(tokenized_contexts[0]['input_ids'][0], skip_special_tokens=True)

    input_data = {
        "context": context,
        "question": summarized_question
    }

    answer = chain.invoke(input_data)
    return answer

# 전체 프로세스 실행 함수
def run_pipeline(file_path, question, llm_base_url="http://localhost:1234/v1"):
    # PDF 로드
    pages = load_pdf(file_path)

    # KoBART 모델 및 토크나이저 로드
    tokenizer = PreTrainedTokenizerFast.from_pretrained("EbanLee/kobart-summary-v3")
    bart_model = BartForConditionalGeneration.from_pretrained("EbanLee/kobart-summary-v3")

    # 텍스트 분할
    docs = split_text(pages, tokenizer)

    # RAG 시스템 및 체인 설정
    llm, emb, rag_chain = setup_rag_system(llm_base_url)

    # 문서 저장소 및 검색기 설정
    vectorstore, retriever = setup_vectorstore(docs, emb)

    # KeyBERT 모델 로드
    keyword_model = KeyBERT()

    # 질문 처리 및 답변 생성
    answer = process_question(question, retriever, tokenizer, bart_model, keyword_model, rag_chain, docs)

    return answer

# 3. 답변 생성

In [27]:
answers = []

for index, row in score_df.iterrows():
    # 각 행의 파일명과 질문을 사용하여 run_pipeline 실행
    file_path = row['target_file_name']
    question = row['question']
    
    # run_pipeline 함수 호출
    answers.append(run_pipeline(file_path=file_path, question=question))

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', 

KeyboardInterrupt: 

In [50]:
my_model = pd.DataFrame(answers)
my_model.columns = ['answer']
my_model.to_csv("file/my_model.csv")

In [4]:
score_df_test = score_df.iloc[18:]
score_df_test

,domain,question,target_answer,target_file_name,target_page_no,context_type,alli_gpt-4-turbo_answer,alli_gpt-4-turbo_ox,alli_gpt-4_answer,alli_gpt-4_ox,...,anything_llm_claude3-opus_answer,anything_llm_claude3-opus_ox,upstage_gpt-3.5-turbo_answer,upstage_gpt-3.5-turbo_ox,upstage_gpt-4-turbo_answer,upstage_gpt-4-turbo_ox,upstage_gpt-4_answer,upstage_gpt-4_ox,upstage_solar_answer,upstage_solar_ox
18,finance,"간병 종사자 등의 인재확보를 위한 처우개선에 관한 법률은 언제 제정되었고, 그 법률...",간병 종사자 등의 인재확보를 위한 처우개선에 관한 법률은 2008년 5월 28일에 ...,130292099630937500_KIFVIP2013-10.pdf,46,table,간병 종사자 등의 인재확보를 위한 처우개선에 관한 법률은 2008년 5월 28일에 ...,O,간병 종사자 등의 인재확보를 위한 처우개선에 관한 법률은 2008년 5월 28일에 ...,O,...,제공해주신 자료에는 간병 종사자 등의 인재확보를 위한 처우개선에 관한 법률의 제정 ...,X,간병 종사자 등의 인재확보를 위한 처우개선에 관한 법률은 2008년 5월 28일에 ...,O,간병 종사자 등의 인재확보를 위한 처우개선에 관한 법률은 2008년 5월 28일에 ...,O,간병 종사자 등의 인재확보를 위한 처우개선에 관한 법률은 2008년 5월 28일에 ...,O,간병 종사자 등의 인재확보를 위한 처우개선에 관한 법률은 2008년 5월 28일에 ...,O
19,finance,1994년부터 2001년까지 고령사회 대책과 관련하여 가장 먼저 이루어진 이벤트와 ...,1994년부터 2001년까지 고령사회 대책과 관련하여 가장 먼저 이루어진 이벤트는 ...,130292099630937500_KIFVIP2013-10.pdf,47,table,1994년부터 2001년까지 고령사회 대책과 관련하여 가장 먼저 이루어진 이벤트는 ...,O,1994년부터 2001년까지 고령사회 대책과 관련하여 가장 먼저 이루어진 이벤트는 ...,O,...,안타깝게도 제공해주신 자료에는 1994년부터 2001년 사이에 있었던 고령사회 대책...,X,"1994년에는 <고령사회 대책 기본법>의 시행이 이루어졌고, 2001년에는 [고령사...",O,1994년부터 2001년까지 고령사회 대책과 관련하여 가장 먼저 이루어진 이벤트는 ...,O,"가장 먼저 이루어진 이벤트는 '고령사회 대책 기본법'의 시행으로, 이는 1994년 ...",O,1994년부터 2001년까지 고령사회 대책과 관련하여 가장 먼저 이루어진 이벤트는 ...,O
20,finance,"1996년, 2001년, 2012년의 대책대강에서 공통적으로 포함되어 있던 고령자의...",1996년에는 고령자의 고용 및 취업 기회 확보를 중점으로 한 대책이 포함되어 있었...,130292099630937500_KIFVIP2013-10.pdf,48,table,"1996년, 2001년, 2012년의 대책대강에서 공통적으로 포함되어 있던 고령자의...",O,"1996년, 2001년, 2012년의 대책대강에서 공통적으로 포함되어 있던 고령자의...",O,...,"안타깝게도 제공해주신 자료에는 1996년, 2001년, 2012년의 고령사회 대책대...",X,"1996, 2001, 2012년 대책대강에서는 모두 고령자의 고용과 취업 기회를 확...",O,"1996년, 2001년, 2012년의 대책대강에서는 고령자의 취업과 관련하여 고용 ...",O,"1996년, 2001년, 2012년의 대책대강에서 고령자의 취업에 관련된 주요 항목...",O,"1996년, 2001년, 2012년의 대책대강에서 공통적으로 포함되어 있던 고령자의...",O
21,finance,"정부, 기업, 금융산업이 협력하여 녹색금융을 확대 및 활성화함으로써 얻을 수 있는 ...","정부, 기업, 금융산업이 협력하여 녹색금융을 확대하고 활성화함으로써 얻을 수 있는 ...",133178946057443204_WP22-05.pdf,21,paragraph,"정부, 기업, 금융산업이 협력하여 녹색금융을 확대 및 활성화함으로써 얻을 수 있는 ...",O,"정부, 기업, 금융산업이 협력하여 녹색금융을 확대 및 활성화함으로써 얻을 수 있는 ...",O,...,"제공해주신 자료에 따르면 정부, 기업, 금융산업이 협력하여 녹색금융을 확대 및 활성...",X,"정부, 기업, 금융산업이 협력하여 녹색금융을 확대 및 활성화함으로써 얻을 수 있는 ...",O,"정부, 기업, 금융산업이 협력하여 녹색금융을 확대 및 활성화함으로써 얻을 수 있는 ...",O,"정부, 기업, 금융산업이 협력하여 녹색금융을 확대 및 활성화함으로써 얻을 수 있는 ...",O,"정부, 기업, 금융산업이 협력하여 녹색금융을 확대 및 활성화함으로써 얻을 수 있는 ...",O
22,finance,주요국들이 지구 평균기온 상승폭을 1.5oC 이내로 억제하기 위한 노력을 강화함에 ...,주요 국가들은 지구 평균 기온 상승폭을 1.5oC 이내로 억제하기 위해 2030년까...,133178946057443204_WP22-05.pdf,23,paragraph,주요국들이 지구 평균기온 상승폭을 1.5oC 이내로 억제하기 위해 필요한 탄소배출량...,O,주요국들이 지구 평균기온 상승폭을 1.5oC 이내로 억제하기 위한 노력을 강화하고 ...,O,...,"제공된 자료에 따르면, 주요국들은 지구 평균기온 상승폭을 1.5°C 이내로 억제하기...",O,주요국들이 지구 평균기온 상승폭을 1.5℃ 이내로 억제하기 위해 2030년까지 탄소...,O,주요국들은 지구 평균기온 상승폭을 1.5oC 이내로 제한하기 위해 2030년까지 2...,O,주요국들이 지구 평균기온 상승폭을 1.5oC 이내로 억제하기 위한 노력을 강화하는 ...,O,주요국들이 지구 평균기온 상승폭을 1.5oC 이내로 억제하기 위해 필요한 탄소배출량...,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,commerce,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율과...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,중국21-18호 2021 라이브커머스 산업 보고.pdf,12,paragraph,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년에 기업의 자체 라이브 방송 거래액은 전체 생방송 라이브커머스 대비 32....,O,...,2020년 기업의 자체 라이브 방송 거래액이 전체 라이브커머스에서 차지하는 비중은 ...,O,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년에 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 ...,O,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율은...,O
296,commerce,"콰이서우와 틱톡에서 거래비율이 급성장한 연도와 그 이유, 그리고 현재 거래액은 어떻...",2020년 콰이서우와 틱톡의 거래비율이 급성장 했습니다. 이는 두 플랫폼이 브랜드 ...,중국21-18호 2021 라이브커머스 산업 보고.pdf,16,paragraph,콰이서우와 틱톡에서 거래비율이 급성장한 연도는 2020년입니다. 이들 플랫폼은 브랜...,O,2020년에 콰이서우와 틱톡 플랫폼 내부 거래비율이 급성장하였습니다. 이는 두 플랫...,O,...,콰이쇼우(Kuaishou)와 틱톡(Douyin)은 중국의 대표적인 숏클립 플랫폼으로...,O,"2020년에

In [5]:
answers_2 = []

for index, row in score_df_test.iterrows():
    # 각 행의 파일명과 질문을 사용하여 run_pipeline 실행
    file_path = row['target_file_name']
    question = row['question']
    
    # run_pipeline 함수 호출
    answers_2.append(run_pipeline(file_path=file_path, question=question))

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map

KeyboardInterrupt: 

In [7]:
my_model_2 = pd.DataFrame(answers_2)
my_model_2.columns = ['answer']
my_model_2.to_csv("file/my_model_2.csv")

In [4]:
score_df_test = score_df.iloc[18 + 47:]
score_df_test

,domain,question,target_answer,target_file_name,target_page_no,context_type,alli_gpt-4-turbo_answer,alli_gpt-4-turbo_ox,alli_gpt-4_answer,alli_gpt-4_ox,...,anything_llm_claude3-opus_answer,anything_llm_claude3-opus_ox,upstage_gpt-3.5-turbo_answer,upstage_gpt-3.5-turbo_ox,upstage_gpt-4-turbo_answer,upstage_gpt-4-turbo_ox,upstage_gpt-4_answer,upstage_gpt-4_ox,upstage_solar_answer,upstage_solar_ox
65,public,교당 평균 지원단가는 23년도와 24년도 사이에 어느 정도 변화하였나요? 그리고 이...,"교당 평균 지원단가는 23년도에서 24년도로 넘어가면서 30.8억 원이 증가하였고,...",2024년+국립대학+육성사업+기본계획.pdf,6,table,"2023년도 교당 평균 지원단가는 123.5억 원이었고, 2024년도에는 154.3...",O,"2023년에는 교당 평균 지원단가가 123.5억 원이었고, 2024년에는 154.3...",O,...,제공해주신 자료에는 2023년과 2024년의 교당 평균 지원단가에 대한 정보가 포함...,X,교당 평균 지원단가는 23년에서 24년으로 증가했습니다. 이 변화율은 30.8억 원...,X,2023년과 2024년 사이에 교당 평균 지원단가는 123.5억 원에서 154.3억...,O,교당 평균 지원단가는 23년도에 123.5억 원에서 24년도에 154.3억 원으로 ...,O,"교당 평균 지원단가는 23년도에는 123.5억 원이었고, 24년도에는 154.3억 ...",X
66,public,"사서교사 정원이 2019년부터 2024년까지 어느 년도에서 가장 많이 증원되었는지,...","2021년과 2022년에 가장 많이 증원되었고, 증원규모는 200이다.",제4차(24_28)+학교도서관+진흥+기본계획.pdf,18,table,"사서교사 정원은 2021년에 가장 많이 증원되었으며, 그 증원 규모는 200명입니다...",O,"2019년부터 2024년까지 사서교사의 정원은 매년 증가하였으며, 가장 많이 증원된...",O,...,죄송합니다. 제공해주신 자료에는 2019년부터 2024년까지의 사서교사 정원 증원 ...,X,2019년부터 2024년까지 사서교사 정원이 가장 많이 증원된 년도는 2021년이며...,O,2019년부터 2024년까지 사서교사 정원이 가장 많이 증원된 년도는 2021년과 ...,O,2019년부터 2024년까지 사서교사 정원이 가장 많이 증원된 년도는 2021년과 ...,O,2019년부터 2024년까지 사서교사 정원이 가장 많이 증원된 연도는 2020년입니...,X
67,public,중앙재난심리회복지원단에서 제공하는 서비스를 각각 이용한 사람들의 수를 각기 다른 부...,유가족·부상자·재난현장경험자 등 트라우마센터나 정신건강복지센터(복지부)의 서비스를 ...,국가안전시스템 개편 종합대책 대국민 보고(12.31. 기준).pdf,11,table,중앙재난심리회복지원단에서 제공하는 서비스를 이용한 사람들의 수는 다음과 같이 부처 ...,O,중앙재난심리회복지원단에서 제공하는 서비스를 이용한 사람들의 수는 다음과 같습니다. ...,O,...,제공해주신 자료에는 중앙재난심리회복지원단의 서비스 이용 현황이나 부처별 이용자 수에...,X,소방 공무원이 중앙재난심리회복지원단의 서비스를 이용한 사람들 중 가장 많은 수를 차...,X,"유가족·부상자·재난현장경험자 등 7,388명이 중앙재난심리회복지원단의 서비스를 이용...",O,중앙재난심리회복지원단에서 제공하는 서비스를 이용한 사람들의 수는 다음과 같습니다: ...,O,중앙재난심리회복지원단에서 제공하는 서비스를 각각 이용한 사람들의 수를 각기 다른 부...,X
68,public,앙골라의 다이아몬드 생산량이 가장 높았던 년도와 그 년도의 생산량은 무엇인가요?,"2023년, 980",외교부-2024년 앙골라개황(저).pdf,26,table,"앙골라의 다이아몬드 생산량이 가장 높았던 년도는 2023년이며, 그 해의 생산량은 ...",O,"앙골라의 다이아몬드 생산량이 가장 높았던 년도는 2018년이며, 그 때의 생산량은 ...",X,...,"죄송합니다만, 제공해주신 자료에는 앙골라의 다이아몬드 생산량에 대한 정보가 전혀 포...",X,"2022년에 앙골라는 875만 캐럿의 다이아몬드를 생산했으며, 이는 해당 국가에서 ...",X,"앙골라의 다이아몬드 생산량이 가장 높았던 년도는 2023년이며, 그 해의 생산량은 ...",O,"앙골라의 다이아몬드 생산량이 가장 높았던 년도는 2018년이며, 그 년도의 생산량은...",X,"앙골라의 다이아몬드 생산량이 가장 높았던 년도는 2018년으로, 그 해의 생산량은 ...",X
69,public,"2019년에 비해 2022년의 무역 수지는 어떻게 변화했으며, 그 원인은 무엇일까요?","2019년의 무역 수지는 223이었지만, 2022년에는 118로 감소했습니다. 이것...",외교부-2024년 앙골라개황(저).pdf,38,table,2019년 대비 2022년의 무역 수지는 223에서 118로 감소했습니다[2]. 이...,O,"2019년의 무역 수지는 223이었고, 2022년의 무역 수지는 118로 감소하였습...",O,...,제공된 자료에 따르면 2022년 앙골라의 무역수지는 327억 4천만 불 흑자를 기록...,X,2022년의 무역 수지는 2019년보다 훨씬 더 나은 상황으로 327억 4천만 불의...,X,"2022년 앙골라의 무역 수지는 327억 4천만 불 흑자를 기록했으며, 이는 전년 ...",X,2022년 앙골라의 무역 수지는 전년 대비 증가하였습니다. 수출은 10% 증가하였고...,X,"2019년과 2022년 사이의 무역 수지 변화는 2022년 앙골라 통계청에 따르면,...",X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,commerce,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율과...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,중국21-18호 2021 라이브커머스 산업 보고.pdf,12,paragraph,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년에 기업의 자체 라이브 방송 거래액은 전체 생방송 라이브커머스 대비 32....,O,...,2020년 기업의 자체 라이브 방송 거래액이 전체 라이브커머스에서 차지하는 비중은 ...,O,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년에 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 ...,O,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율은...,O
296,commerce,"콰이서우와 틱톡에서 거래비율이 급성장한 연도와 그 이유, 그리고 현재 거래액은 어떻...",2020년 콰이서우와 틱톡의 거래비율이 급성장 했습니다. 이는 두 플랫폼이 브랜드 ...,중국21-18호 2021 라이브커머스 산업 보고.pdf,16,paragraph,콰이서우와 틱톡에서 거래비율이 급성장한 연도는 2020년입니다. 이들 플랫폼은 브랜...,O,2020년에 콰이서우와 틱톡 플랫폼 내부 거래비율이 급성장하였습니다. 이는 두 플랫...,O,...,콰이쇼우(Kuaishou)와 틱톡(Douyin)은 중국의 대표적인 숏클립 플랫폼으로...,O,"2020년에는 콰이서우와 틱톡의 플랫폼 내부 거래비율이 급성장하여, 현재 거래액의 ...",O,"2020년에 콰이서우와 틱톡 플랫폼 내부의 거래비율이 급성장하였으며, 이는 두 플랫...",O,20

In [5]:
answers_3 = []

for index, row in score_df_test.iterrows():
    # 각 행의 파일명과 질문을 사용하여 run_pipeline 실행
    file_path = row['target_file_name']
    question = row['question']
    
    # run_pipeline 함수 호출
    answers_3.append(run_pipeline(file_path=file_path, question=question))

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map

KeyboardInterrupt: 

In [8]:
len(answers_3)

46

In [9]:
my_model_3 = pd.DataFrame(answers_3)
my_model_3.columns = ['answer']
my_model_3.to_csv("file/my_model_3.csv")

In [13]:
score_df.iloc[18 + 47 + 46:]

,domain,question,target_answer,target_file_name,target_page_no,context_type,alli_gpt-4-turbo_answer,alli_gpt-4-turbo_ox,alli_gpt-4_answer,alli_gpt-4_ox,...,anything_llm_claude3-opus_answer,anything_llm_claude3-opus_ox,upstage_gpt-3.5-turbo_answer,upstage_gpt-3.5-turbo_ox,upstage_gpt-4-turbo_answer,upstage_gpt-4-turbo_ox,upstage_gpt-4_answer,upstage_gpt-4_ox,upstage_solar_answer,upstage_solar_ox
111,public,앙골라의 TAAG항공이 연결한 국내외 노선 개수와 이에 대한 유럽연합의 안전 조치가...,"TAAG항공은 국내외 주요 도시를 연결하고 있으며, 국제선 13개 노선과 국내선 1...",외교부-2024년 앙골라개황(저).pdf,23,paragraph,앙골라의 TAAG항공은 국제선 13개 노선과 국내선 12개 노선을 연결하고 있습니다...,O,앙골라의 TAAG항공은 국제선 13개 노선과 국내선 12개 노선을 연결하고 있습니다...,O,...,"제공된 자료에는 앙골라 TAAG항공의 국내외 노선 개수, 이에 대한 유럽연합의 안전...",X,앙골라의 TAAG항공은 국내외 주요 도시를 연결하며 국제선 13개 노선과 국내선 1...,O,앙골라의 TAAG항공은 국제선 13개 노선과 국내선 12개 노선을 운영하고 있습니다...,O,"앙골라의 TAAG항공은 국내외 주요 도시를 연결하며 국제선 13개 노선, 국내선 1...",O,TAAG 앙골라 항공은 국내외 13개 국제선과 12개 국내선을 연결합니다. 유럽 연...,O
112,public,수도 루안다를 중심으로 구축된 앙골라의 주요 교통 인프라와 그것이 앙골라 경제에 미...,"앙골라에는 총 2,852km의 철도와 51,429km 규모의 도로망이 루안다를 중심...",외교부-2024년 앙골라개황(저).pdf,23,paragraph,"앙골라의 수도 루안다를 중심으로 구축된 주요 교통 인프라에는 루안다 신국제공항, 철...",O,"앙골라의 주요 교통 인프라는 항공, 항만, 철도, 도로 등으로 구성되어 있습니다. ...",O,...,"죄송합니다만, 제공해주신 자료에는 앙골라 수도 루안다의 주요 교통 인프라 현황이나 ...",X,"앙골라는 수도 루안다를 중심으로 각 지방 주요 도시를 연결하는 51,429km 규모...",O,"앙골라의 주요 교통 인프라는 수도 루안다를 중심으로 하는 광범위한 도로망, 철도 시...",O,"앙골라의 교통 인프라는 수도 루안다를 중심으로 구축되어 있으며, 철도, 도로, 항공...",O,"앙골라의 주요 교통 인프라는 수도 루안다를 중심으로 구축되어 있으며, 철도, 도로,...",O
113,public,"앙골라 정부의 외국인투자 유치법 개정에는 어떤 주요한 변화가 있었고, 그 변화로 인...","앙골라 정부는 외국 투자유치를 위해 1979년 외국인투자유치법을 제정하고, 이를 여...",외교부-2024년 앙골라개황(저).pdf,24,paragraph,앙골라 정부는 외국인투자 유치를 위해 여러 차례에 걸쳐 외국인투자유치법을 개정하였습...,O,"앙골라 정부는 외국인투자유치를 위해 1979년 외국인투자유치법을 제정하고, 이후 1...",O,...,제공된 자료에는 앙골라 정부의 외국인투자 유치법 개정 내용이나 그에 따른 앙골라 경...,X,"앙골라 정부는 외국 투자유치를 위해 1979년 외국인투자유치법을 제정하고, 1988...",X,"앙골라 정부는 외국인투자유치법에서 투자 승인 주체 변경, 최소 투자금액 폐지, 현지...",O,"앙골라의 외국인투자 유치법 개정은 투자 승인 주체 변경, 최소 투자금액 폐지, 현지...",O,"앙골라 정부의 외국인투자 유치법 개정에는 투자 승인 주체 변경, 최소 투자금액 폐지...",O
114,public,"앙골라의 2022년 기준 원유 확인 매장량과 일산량, 주요 원유 수출 대상국을 각각...","2022년 기준 앙골라의 원유 확인 매장량은 25억 5천 만 배럴이며, 일산량은 1...",외교부-2024년 앙골라개황(저).pdf,26,paragraph,"앙골라의 2022년 기준 원유 확인 매장량은 25억 5천 만 배럴이며, 일산량은 1...",O,"2022년 기준 앙골라의 원유 확인 매장량은 25억 5천만 배럴이며, 일산량은 11...",O,...,"제공된 자료에는 앙골라의 2022년 원유 확인 매장량, 일일 생산량, 주요 수출 대...",X,"2022년 앙골라의 원유 확인 매장량은 25억 5천만 배럴이며, 주요 원유 수출 대...",O,"2022년 기준 앙골라의 원유 확인 매장량은 25억 5천 만 배럴이며, 일산량은 1...",O,"2022년 기준 앙골라의 원유 확인 매장량은 25억 5천 만 배럴이며, 일산량은 1...",O,"앙골라의 2022년 기준 원유 확인 매장량은 25억 5천만 배럴이며, 일산량은 11...",O
115,public,앙골라의 소득 불균형 지표는 어떠한가? 그리고 UNDP 인간개발지수(HDI) 조사 ...,"상위 20%가 전체 소득의 63%, 하위 20%가 전체 소득의 2%로, 상위 20%...",외교부-2024년 앙골라개황(저).pdf,33,paragraph,앙골라의 소득 불균형은 상당히 심각한 수준입니다. 상위 20%가 전체 소득의 63%...,O,앙골라의 소득 불균형은 상당히 심각한 상태입니다. 상위 20%의 인구가 전체 소득의...,O,...,제공된 자료에는 앙골라의 소득 불균형 수준이나 UNDP 인간개발지수(HDI) 조사 ...,X,"앙골라의 소득 불균형 지표는 상위 20%가 전체 소득의 63%, 하위 20%가 전체...",O,"앙골라의 소득 불균형은 매우 심각하며, 상위 20%가 전체 소득의 63%를 차지하고...",O,앙골라의 소득 불균형은 상당히 심각합니다. 상위 20%의 소득이 전체 소득의 63%...,O,"앙골라의 소득 불균형 지표는 상위 20%가 전체 소득의 63%를 차지하고, 하위 2...",O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,commerce,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율과...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,중국21-18호 2021 라이브커머스 산업 보고.pdf,12,paragraph,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년에 기업의 자체 라이브 방송 거래액은 전체 생방송 라이브커머스 대비 32....,O,...,2020년 기업의 자체 라이브 방송 거래액이 전체 라이브커머스에서 차지하는 비중은 ...,O,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년에 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 ...,O,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율은...,O
296,commerce,"콰이서우와 틱톡에서 거래비율이 급성장한 연도와 그 이유, 그리고 현재 거래액은 어떻...",2020년 콰이서우와 틱톡의 거래비율이 급성장 했습니다. 이는 두 플랫폼이 브랜드 ...,중국21-18호 2021 라이브커머스 산업 보고.pdf,16,paragraph,콰이서우와 틱톡에서 거래비율이 급성장한 연도는 2020년입니다. 이들 플랫폼은 브랜...,O,2020년에 콰이서우와 틱톡 플랫폼 내부 거래비율이 급성장하였습니다. 이는 두 플랫...,O,...,콰이쇼우(Kuaishou)와 틱톡(Douyin)은 중국의 대표적인 숏클립 플랫폼으로...,O,"2020년에는 콰이서우와 틱톡의 플랫폼 내부 거래비율이 급

In [14]:
score_df_test = score_df.iloc[18 + 47 + 46:]
score_df_test

,domain,question,target_answer,target_file_name,target_page_no,context_type,alli_gpt-4-turbo_answer,alli_gpt-4-turbo_ox,alli_gpt-4_answer,alli_gpt-4_ox,...,anything_llm_claude3-opus_answer,anything_llm_claude3-opus_ox,upstage_gpt-3.5-turbo_answer,upstage_gpt-3.5-turbo_ox,upstage_gpt-4-turbo_answer,upstage_gpt-4-turbo_ox,upstage_gpt-4_answer,upstage_gpt-4_ox,upstage_solar_answer,upstage_solar_ox
111,public,앙골라의 TAAG항공이 연결한 국내외 노선 개수와 이에 대한 유럽연합의 안전 조치가...,"TAAG항공은 국내외 주요 도시를 연결하고 있으며, 국제선 13개 노선과 국내선 1...",외교부-2024년 앙골라개황(저).pdf,23,paragraph,앙골라의 TAAG항공은 국제선 13개 노선과 국내선 12개 노선을 연결하고 있습니다...,O,앙골라의 TAAG항공은 국제선 13개 노선과 국내선 12개 노선을 연결하고 있습니다...,O,...,"제공된 자료에는 앙골라 TAAG항공의 국내외 노선 개수, 이에 대한 유럽연합의 안전...",X,앙골라의 TAAG항공은 국내외 주요 도시를 연결하며 국제선 13개 노선과 국내선 1...,O,앙골라의 TAAG항공은 국제선 13개 노선과 국내선 12개 노선을 운영하고 있습니다...,O,"앙골라의 TAAG항공은 국내외 주요 도시를 연결하며 국제선 13개 노선, 국내선 1...",O,TAAG 앙골라 항공은 국내외 13개 국제선과 12개 국내선을 연결합니다. 유럽 연...,O
112,public,수도 루안다를 중심으로 구축된 앙골라의 주요 교통 인프라와 그것이 앙골라 경제에 미...,"앙골라에는 총 2,852km의 철도와 51,429km 규모의 도로망이 루안다를 중심...",외교부-2024년 앙골라개황(저).pdf,23,paragraph,"앙골라의 수도 루안다를 중심으로 구축된 주요 교통 인프라에는 루안다 신국제공항, 철...",O,"앙골라의 주요 교통 인프라는 항공, 항만, 철도, 도로 등으로 구성되어 있습니다. ...",O,...,"죄송합니다만, 제공해주신 자료에는 앙골라 수도 루안다의 주요 교통 인프라 현황이나 ...",X,"앙골라는 수도 루안다를 중심으로 각 지방 주요 도시를 연결하는 51,429km 규모...",O,"앙골라의 주요 교통 인프라는 수도 루안다를 중심으로 하는 광범위한 도로망, 철도 시...",O,"앙골라의 교통 인프라는 수도 루안다를 중심으로 구축되어 있으며, 철도, 도로, 항공...",O,"앙골라의 주요 교통 인프라는 수도 루안다를 중심으로 구축되어 있으며, 철도, 도로,...",O
113,public,"앙골라 정부의 외국인투자 유치법 개정에는 어떤 주요한 변화가 있었고, 그 변화로 인...","앙골라 정부는 외국 투자유치를 위해 1979년 외국인투자유치법을 제정하고, 이를 여...",외교부-2024년 앙골라개황(저).pdf,24,paragraph,앙골라 정부는 외국인투자 유치를 위해 여러 차례에 걸쳐 외국인투자유치법을 개정하였습...,O,"앙골라 정부는 외국인투자유치를 위해 1979년 외국인투자유치법을 제정하고, 이후 1...",O,...,제공된 자료에는 앙골라 정부의 외국인투자 유치법 개정 내용이나 그에 따른 앙골라 경...,X,"앙골라 정부는 외국 투자유치를 위해 1979년 외국인투자유치법을 제정하고, 1988...",X,"앙골라 정부는 외국인투자유치법에서 투자 승인 주체 변경, 최소 투자금액 폐지, 현지...",O,"앙골라의 외국인투자 유치법 개정은 투자 승인 주체 변경, 최소 투자금액 폐지, 현지...",O,"앙골라 정부의 외국인투자 유치법 개정에는 투자 승인 주체 변경, 최소 투자금액 폐지...",O
114,public,"앙골라의 2022년 기준 원유 확인 매장량과 일산량, 주요 원유 수출 대상국을 각각...","2022년 기준 앙골라의 원유 확인 매장량은 25억 5천 만 배럴이며, 일산량은 1...",외교부-2024년 앙골라개황(저).pdf,26,paragraph,"앙골라의 2022년 기준 원유 확인 매장량은 25억 5천 만 배럴이며, 일산량은 1...",O,"2022년 기준 앙골라의 원유 확인 매장량은 25억 5천만 배럴이며, 일산량은 11...",O,...,"제공된 자료에는 앙골라의 2022년 원유 확인 매장량, 일일 생산량, 주요 수출 대...",X,"2022년 앙골라의 원유 확인 매장량은 25억 5천만 배럴이며, 주요 원유 수출 대...",O,"2022년 기준 앙골라의 원유 확인 매장량은 25억 5천 만 배럴이며, 일산량은 1...",O,"2022년 기준 앙골라의 원유 확인 매장량은 25억 5천 만 배럴이며, 일산량은 1...",O,"앙골라의 2022년 기준 원유 확인 매장량은 25억 5천만 배럴이며, 일산량은 11...",O
115,public,앙골라의 소득 불균형 지표는 어떠한가? 그리고 UNDP 인간개발지수(HDI) 조사 ...,"상위 20%가 전체 소득의 63%, 하위 20%가 전체 소득의 2%로, 상위 20%...",외교부-2024년 앙골라개황(저).pdf,33,paragraph,앙골라의 소득 불균형은 상당히 심각한 수준입니다. 상위 20%가 전체 소득의 63%...,O,앙골라의 소득 불균형은 상당히 심각한 상태입니다. 상위 20%의 인구가 전체 소득의...,O,...,제공된 자료에는 앙골라의 소득 불균형 수준이나 UNDP 인간개발지수(HDI) 조사 ...,X,"앙골라의 소득 불균형 지표는 상위 20%가 전체 소득의 63%, 하위 20%가 전체...",O,"앙골라의 소득 불균형은 매우 심각하며, 상위 20%가 전체 소득의 63%를 차지하고...",O,앙골라의 소득 불균형은 상당히 심각합니다. 상위 20%의 소득이 전체 소득의 63%...,O,"앙골라의 소득 불균형 지표는 상위 20%가 전체 소득의 63%를 차지하고, 하위 2...",O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,commerce,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율과...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,중국21-18호 2021 라이브커머스 산업 보고.pdf,12,paragraph,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년에 기업의 자체 라이브 방송 거래액은 전체 생방송 라이브커머스 대비 32....,O,...,2020년 기업의 자체 라이브 방송 거래액이 전체 라이브커머스에서 차지하는 비중은 ...,O,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년에 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 ...,O,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율은...,O
296,commerce,"콰이서우와 틱톡에서 거래비율이 급성장한 연도와 그 이유, 그리고 현재 거래액은 어떻...",2020년 콰이서우와 틱톡의 거래비율이 급성장 했습니다. 이는 두 플랫폼이 브랜드 ...,중국21-18호 2021 라이브커머스 산업 보고.pdf,16,paragraph,콰이서우와 틱톡에서 거래비율이 급성장한 연도는 2020년입니다. 이들 플랫폼은 브랜...,O,2020년에 콰이서우와 틱톡 플랫폼 내부 거래비율이 급성장하였습니다. 이는 두 플랫...,O,...,콰이쇼우(Kuaishou)와 틱톡(Douyin)은 중국의 대표적인 숏클립 플랫폼으로...,O,"2020년에는 콰이서우와 틱톡의 플랫폼 내부 거래비율이 급

In [ ]:
answers_4 = []

for index, row in score_df_test.iterrows():
    # 각 행의 파일명과 질문을 사용하여 run_pipeline 실행
    file_path = row['target_file_name']
    question = row['question']
    
    # run_pipeline 함수 호출
    answers_4.append(run_pipeline(file_path=file_path, question=question))